In [1]:
import re
import pandas as pd

/var/folders/gt/sg3v8rd13l52jx91mfbgzbfc0000gn/T/ipykernel_37895/1566173301.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open("ezkl_settings.out", 'r') as f:
    content = f.read()
    rows = re.findall(r'model uses (\d+) rows', content)

In [3]:
params = [2546240, 9188480, 34760960, 34760960, 859520964]

In [4]:
df = pd.DataFrame({'params': params})

df['rows'] = [int(row) for row in rows] + [None]

In [5]:
df['params_pct'] = df['params'] / df['params'].iloc[-1]

In [6]:
df['row_estimate'] = df['rows'] / df['params_pct']

In [7]:
df['row_estimate'].iloc[-1] = df['row_estimate'].iloc[:-1].mean()

/var/folders/gt/sg3v8rd13l52jx91mfbgzbfc0000gn/T/ipykernel_37895/2349657401.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['row_estimate'].iloc[-1] = df['row_estimate'].iloc[:-1].mean()
/var/folders/gt/sg3v8rd13l52jx91mfbgzbfc0000gn/

In [8]:
bench = pd.read_csv("../mnist/bench/bench.csv", index_col=0)
bench

,circom_time,circom_mem,ezkl_time,ezkl_mem,model_params,circom_constrs,ezkl_rows,opml_time,circom_op_time,ezkl_op_time
1,0 days 00:04:38.800000,85446120,0 days 00:00:33.200000,2869864,320,11378432,184408,0 days 00:00:00.040334380,0 days 00:04:38.840334380,0 days 00:00:33.240334380
2,0 days 00:08:44.180000,129358852,0 days 00:00:36.450000,2857136,320,16867552,224968,0 days 00:00:00.039514066,0 days 00:08:44.219514066,0 days 00:00:36.489514066
3,0 days 00:10:16.600000,136911572,0 days 00:03:08.110000,22516096,18816,23101472,1373720,0 days 00:00:00.039434895,0 days 00:10:16.639434895,0 days 00:03:08.149434895
4,0 days 00:10:50.140000,138164812,0 days 00:03:08.130000,22510672,18816,24725472,1385048,0 days 00:00:00.039542312,0 days 00:10:50.179542312,0 days 00:03:08.169542312
7,0 days 00:11:04.880000,137595576,0 days 00:03:11.770000,22512552,34826,24741472,1392273,0 days 00:00:00,0 days 00:11:04.880000,0 days 00:03:11.770000


In [9]:
# use scipy linear regression on ezkl_time vs ezkl_rows and ezkl_mem vs ezkl_rows

from scipy.stats import linregress

time = linregress(bench['ezkl_rows'], bench['ezkl_time'] / pd.Timedelta('1h'))
mem = linregress(bench['ezkl_rows'], bench['ezkl_mem'])


In [10]:
# apply the linear regression to the row estimate


df['mem_estimate'] = (mem.intercept + mem.slope * df['rows']) / 1e9
df['time_estimate'] = time.intercept + time.slope * df['rows']

df['mem_estimate'].iloc[-1] = (mem.intercept + mem.slope * df['row_estimate'].iloc[-1]) / 1e9
df['time_estimate'].iloc[-1] = time.intercept + time.slope * df['row_estimate'].iloc[-1] 
df

/var/folders/gt/sg3v8rd13l52jx91mfbgzbfc0000gn/T/ipykernel_37895/3059676659.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['mem_estimate'].iloc[-1] = (mem.intercept + mem.slope * df['row_estimate'].iloc[-1]) / 1e9
/var/folders/gt/sg3

,params,rows,params_pct,row_estimate,mem_estimate,time_estimate
0,2546240,1.082124e+10,0.002962,3.652870e+12,180.241362,393.877512
1,9188480,5.107530e+09,0.010690,4.777753e+11,85.072026,185.907839
2,34760960,4.429483e+09,0.040442,1.095261e+11,73.778273,161.228059
3,34760960,1.147973e+09,0.040442,2.838550e+10,19.120456,41.786559
4,859520964,NaN,1.000000,1.067139e+12,17774.596266,38842.104835
